<a href="https://colab.research.google.com/github/kazuki-de/git_reserch_code/blob/master/AAE_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0rc0
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation , Softmax, MaxPool2D,UpSampling2D, Input, BatchNormalization, add, AveragePooling2D,InputLayer
from tensorflow.keras import Model,Sequential
import datetime
#import tensorflow as tf
import numpy as np
import pandas as pd
#from PIL import Image
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow_hub as hub
 
from tensorflow.keras import layers
import os
import time

import glob
import matplotlib.pyplot as plt
import PIL
import imageio

from IPython import display

     |████████████████████████████████| 380.5MB 38kB/s 
     |████████████████████████████████| 501kB 31.9MB/s 
     |████████████████████████████████| 4.3MB 27.9MB/s 


In [2]:
mnist = tf.keras.datasets.mnist
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#x_train  = x_train.reshape(60000, 784)
#x_test   = x_test.reshape(10000, 784)
#x_train  = x_train.astype('float32')/255.0
#x_test   = x_test.astype('float32')/255.0

# Add a channels dimension
#x_train = x_train[..., tf.newaxis]
#x_test = x_test[..., tf.newaxis]
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).batch(100)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(100)

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
TRAIN_BUF = 60000
BATCH_SIZE = 100

TEST_BUF = 10000

In [0]:
class Encode(Model):
  def __init__(self,dimension):
    super(Encode, self).__init__()
    self.conv1 = Conv2D(8,(7,7),padding="same", activation='relu', name='Conv1')
    self.pool1 = MaxPool2D((2, 2), padding='same', name='Maxpool1')
    self.conv2 = Conv2D(16,(5,5), padding="same",activation='relu', name='Conv2')
    self.pool2 = MaxPool2D((2, 2), padding='same', name='Maxpool2')
    self.conv3 = Conv2D(16,(3,3),padding="same", activation='relu', name='Conv3')
    self.pool3 = MaxPool2D((2, 2), padding='same', name='Maxpool3')
    self.Dense = Dense(dimention)
    
  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.conv3(x)
    x = self.pool3(x)
    return self.Dense(x)


    

In [0]:
class Decode(Model):
  def __init__(self,dimension):
    super(Decode, self).__init__()
    self.Dense = Dense(dimension,activation="relu")
    self.Reshape = Reshape(target_shape=(32, 32, 16))
    self.conv4 = Conv2D(16,(3,3),padding="same", activation='relu', name='Conv4')
    self.uppool1 = UpSampling2D((2, 2), name='Upsumpl1')
    self.conv5 = Conv2D(16,(5,5), padding='same',activation='relu', name='Conv5')
    self.uppool2 = UpSampling2D((2, 2), name='Upsumpl2')
    self.conv6 =  Conv2D(8,(7,7), padding='same',activation='relu', name='Conv6')
    self.uppool3 = UpSampling2D((2, 2), name='Upsumpl3')
    self.conv7 = Conv2D(3,(3,3), padding='same',activation='relu', name='Conv7')
  def call(self, x):
    x = self.Dense(x)
    x = self.Reshape(x)
    x = self.conv4(x)
    x = self.uppool1(x)
    x = self.conv5(x)
    x = self.uppool2(x)
    x = self.conv6(x)
    x = self.uppool3(x)
    return self.conv7(x)

In [0]:
class discriminator_z(Model):
    def __init__(self):
        super(Decode, self).__init__()
        self.dense1 = Dense(1000, activation='relu')
        self.dense2 = Dense(512, activation='relu')
        self.dense3 = Dense(1,, activation='relu')
        self.noise = tf.keras.layers.GaussianNoise(stddev=0.3)
    def call(self, x):
        x = self.noise(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)

In [0]:
class discriminator_y(Model):
    def __init__(self):
        super(Decode, self).__init__()
        self.dense1 = Dense(1000, activation='relu')
        self.dense2 = Dense(512, activation='relu')
        self.dense3 = Dense(1,, activation='relu')
        self.noise = tf.keras.layers.GaussianNoise(stddev=0.3)
    def call(self, x):
        x = self.noise(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)

In [0]:
class AAE(Model):
    def __init__(self, latent_dim,ndim_x=28*28, ndim_y=16, ndim_z=10, ndim_h=1000):
        self.ndim_x = ndim_x
        self.ndim_y = ndim_y
        self.ndim_z = ndim_z
        self.ndim_h = ndim_h
        super(AAE, self).__init__()
        self.latent_dim = latent_dim
        self.Encoder = Encoder()
        self.Decoder = Decoder()
        self.Encoder_y = Dense(ndim_y)
        self.Encoder_z = Dense(ndim_z)
        self.decoder.merge_y = Dense(ndim_h,use_bias=False)
		self.decoder.merge_z = Dense(ndim_h,use_bias=False)
		
        self.discriminator_y = discriminator_y
        self.discriminator_z = discriminator_z
        
    def encode(self, x, apply_softmax = True):
        encode1 = self.Encoder(x)
        encode_y = self.Encoder_y(encode1)
        encode_z = self.Encoder_z(encode1)
        if apply_softmax:
            encode_y = Activation("softmax")(encode_y)
        return encode_y, encode_z
    
    def discriminate_z(self, z, apply_softmax=False):
		logit = self.discriminator_z(z)
		if apply_softmax:
			return Activation("softmax")(logit)
		return logit
    def discriminate_y(self, y, apply_softmax=False):
		logit = self.discriminator_y(y)
		if apply_softmax:
			return Activation("softmax")(logit)
		return logit
    def decode_yz_x(self, y, z):
		merge = self.decoder.merge_y(y) + self.decoder.merge_z(z)
		merge = Activation("relu")(merge)
		return self.Decoder(merge)
    

In [0]:
def mean_squere_error(x,x_reconst):
    reconst_loss = (x_reconst - x) ** 2
    reconst_loss = tf.reduce_mean(reconst_loss, axis=[1,2,3])

In [0]:
def reconstract_loss(x_u):
    y_onehot_u, z_u = model.encode_x_yz(x_u, apply_softmax_y=True)
    x_reconstruction_u = model.decode_yz_x(y_onehot_u, z_u)
    loss_mean = mean_squere_error(x_u,x_reconstruction_u)
    return tf.reduce_mean(loss_mean)       

In [0]:
def adversarial_phase(model,x_u,batchsize):
    
    class_true = np.zeros(batchsize, dtype=np.int32)
	class_fake = np.ones(batchsize, dtype=np.int32)
    y_onehot_fake_u, z_fake_u = model.encode_x_yz(x_u, apply_softmax_y=True)
    z_ture = tf.random.normal(shape=z_fake_u,mean=0.0,stddev=1.0,dtype=tf.dtypes.float32)
    y_onehot_true = 
    #y_onehot_true = sampler.onehot_categorical(args.batchsize, model.ndim_y)

    dz_true = model.discriminate_z(z_true, apply_softmax=True)
    dz_fake = model.discriminate_z(z_fake_u, apply_softmax=True)
    dy_true = model.discriminate_y(y_onehot_true, apply_softmax=True)
    dy_fake = model.discriminate_y(y_onehot_fake_u, apply_softmax=True)

    #discriminator_z_confidence_true = float(xp.mean(F.softmax(dz_true).data[:, 0]))
    #discriminator_z_confidence_fake = float(xp.mean(F.softmax(dz_fake).data[:, 1]))
    #discriminator_y_confidence_true = float(xp.mean(F.softmax(dy_true).data[:, 0]))
    #discriminator_y_confidence_fake = float(xp.mean(F.softmax(dy_fake).data[:, 1]))

    loss_discriminator_z = tf.keras.losses.BinaryCrossentropy(dz_true, class_true) + tf.keras.losses.BinaryCrossentropy(dz_fake, class_fake)
    loss_discriminator_y = tf.keras.losses.BinaryCrossentropy(dy_true, class_true) + tf.keras.losses.BinaryCrossentropy(dy_fake, class_fake)
    loss_discriminator = loss_discriminator_z + loss_discriminator_y
    return loss_discriminator

In [0]:
def generator_phase(model,x_u,batchsize):
    class_true = np.zeros(batchsize, dtype=np.int32)
	class_fake = np.ones(batchsize, dtype=np.int32)
    y_onehot_fake_u, z_fake_u = model.encode_x_yz(x_u, apply_softmax_y=True)

    dz_fake = model.discriminate_z(z_fake_u, apply_softmax=True)
    dy_fake = model.discriminate_y(y_onehot_fake_u, apply_softmax=True)

    loss_generator = tf.keras.losses.BinaryCrossentropy(dz_fake, class_true) + tf.keras.losses.BinaryCrossentropy(dy_fake, class_true)
    return loss_generator


In [0]:
@tf.function
def train_step(model,x_u,batch_size,optimizer1,optimizer2,optimizer3,train_loss):
    with tf.GradientTape() as tape1,tf.GradientTape() as tape2, tf.GradientTape() as tape3:
        loss1 = reconstract_loss(x_u)
        loss2 = adversarial_phase(model,x_u,batchsize)
        loss3 = generator_phase(model,x_u,batchsize)
    gradients1 = tape1.gradient(loss1, model.trainable_variables)
    gradients2 = tape2.gradient(loss2, model.trainable_variables)
    gradients3 = tape3.gradient(loss3, model.trainable_variables)
    optimizer1.apply_gradients(zip(gradients1, model.trainable_variables))
    optimizer2.apply_gradients(zip(gradients2, model.trainable_variables))    
    optimizer3.apply_gradients(zip(gradients3, model.trainable_variables))
    loss = loss1 + loss2 +loss3
    train_loss(loss)

In [0]:
image_shape = (28, 28, 1)
latent_dim = 200
train_loss = tf.keras.metrics.Sum(name='train_loss')
opt1 = tf.keras.optimizers.Adam(1e-4)
opt2 = tf.keras.optimizers.Adam(1e-4)
opt3 = tf.keras.optimizers.Adam(1e-4)
model = AAE()

In [0]:
c_dir = "/content/drive/My Drive/research_/weight_AAE_mnist"
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
manager = tf.train.CheckpointManager(ckpt, c_dir, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)

if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))

In [0]:
epochs = 2000
batch_size = 100
log_path = "/content/drive/My Drive/research_//weight_AAE_mnist/log.csv"

In [0]:
from google.colab import drive
drive.mount("/content/drive")
number = 1
cols = ["epoch","loss","Saved"]
df = pd.DataFrame(index=[],columns=cols)
if(os.path.exists(log_path)):
    df= pd.read_csv(log_path)
    print(log_path+"is exist. loaded csv file")
    number = df["epoch"].max()
    loss_max = df["loss"].max()
    print(loss_max)
print(number)

In [0]:
for epoch in range(1, epochs + 1):
    start_time = time.time()
    for train_x, train_y in train_ds:
        #print(x_l)
        train_step(model,train_x,batch_size,opt1,opt2,opt3,train_loss)
    print(epoch)
    print(train_loss.result())
    if number==1:
        manager.save()
        print("saved")
    if number!=1:
        if df["loss"].max()<=elbo:
            manager.save()
            a = "saved"
            print(a)
        else:
            a = "not saved"
            print(a)
    number+=1
    elbo_np = train_loss.result().numpy()
    record = pd.Series([number,elbo_np,a],index = cols)
    df = df.append(record,ignore_index=True)
    df.to_csv(log_path,index=False)   
    #if epoch%40==0:
        #manager.save()
    
    
    end_time = time.time()